# Dynex SDK - Hello World 

Please copy "dynex.ini" (your configuration file) in the working directory before using this example! The file can be downloaded in your market place, section "INSTALLATON" - "STEP 2" - button "DOWNLOAD".

In [ ]:
import dynex
import dimod
from pyqubo import Array

## Verify Version of Dynex SDK

In [ ]:
dynex.__version__

## Test your configuration

Successful run is required to start sampling with the Dynex SDK

In [ ]:
dynex.test()

## View your API acccount

In [ ]:
dynex.account_status()

## Create a simple QUBO formulation

In [ ]:
N = 15
K = 3
numbers = [4.8097315016016315, 4.325157567810298, 2.9877429101815127,
           3.199880179616316, 0.5787939511978596, 1.2520928214246918,
           2.262867466401502, 1.2300003067401255, 2.1601079352817925,
           3.63753899583021, 4.598232793833491, 2.6215815162575646,
           3.4227134835783364, 0.28254151584552023, 4.2548151473817075]

q = Array.create('q', N, 'BINARY')
H = sum(numbers[i] * q[i] for i in range(N)) + 5.0 * (sum(q) - K)**2
model = H.compile()
Q, offset = model.to_qubo(index_label=True)

In [ ]:
# BQM from QUBO:
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset)

## Sample on Dynex

In [ ]:
model = dynex.BQM(bqm);
sampler = dynex.DynexSampler(model,  mainnet=False, description='Dynex SDK test');
sampleset = sampler.sample(num_reads=5000, annealing_time = 200, debugging=False);
print('Result:')
print(sampleset)

## Draw Sampleset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# generate data:
data = np.zeros(len(sampleset.variables)) 
for s in sampleset.record: #s[0] samples [1] energy [2] num_oc
    for i in range(0,len(s[0])):
        data[i] += ( s[0][i] * s[2] ); # weighted by num_oc

plt.bar(sampleset.variables, data)
plt.bar(sampleset.variables, sampleset.first.sample.values())
plt.xlabel("Variables")
plt.ylabel("Occurences (positive)")
plt.title("Sampleset (orange: lowest ground state)")
plt.show() 